## Angle calibration

Graphs of the MCA measurement for different angles is presented in the following section.

In [30]:
ANGLES_OUTPUT_PATH = r"G:\.shortcut-targets-by-id\1IcR20O5deY8V-HZhJrHf7ZL2cmAiG7GY\ComptonMeasurements\Angles"
MCA_FILE_EXTENSION = ".mca"
STARTING_COUNT_CHANNEL = 568

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import os
import pathlib
import mca_output
import plotly_utils

def find_mca_files_in_folder(folder: str) -> list[pathlib.Path]:
    folder_path = pathlib.Path(folder)
    return [pathlib.Path.joinpath(folder_path, out_file) for out_file in os.listdir(folder) if pathlib.Path(out_file).suffix == MCA_FILE_EXTENSION]

# Find all MCA files in given folder
if not os.path.exists(ANGLES_OUTPUT_PATH) or not os.path.isdir(ANGLES_OUTPUT_PATH):
    raise FileNotFoundError(f"Directory \"{ANGLES_OUTPUT_PATH}\" does not exist")

angle_calibration_mca_files = find_mca_files_in_folder(ANGLES_OUTPUT_PATH)
angle_calibration_mca_outs = [mca_output.parse_output_file(mca_file.absolute()) for mca_file in angle_calibration_mca_files]


In [32]:
for mca_out in angle_calibration_mca_outs:
    fig = plotly_utils.generate_mca_out_figure(mca_out, pathlib.Path(mca_out.path).stem)
    fig.show()

### Extract Maximum Intensity for Each Angle

The following excerpt extracts the bin with the highest count number around the 3000 region.

In [33]:
import numpy as np

for mca_out in angle_calibration_mca_outs:
    channel_count = np.array(mca_out.channel_count_list, np.float32) / mca_out.measurement_time
    max_index = np.argmax(channel_count)
    max_counts = channel_count[max_index]
    print(f"{pathlib.Path(mca_out.path).stem}: channel: {max_index}, counts: {max_counts}")

angle_2.5: channel: 2916, counts: 163.4600067138672
angle_-10: channel: 2844, counts: 12.600000381469727
angle_7.5: channel: 2930, counts: 182.10000610351562
angle_5: channel: 2930, counts: 177.69000244140625
angle_0: channel: 2930, counts: 134.7899932861328
angle_20: channel: 2894, counts: 51.5099983215332
angle_-5: channel: 2891, counts: 64.52999877929688
angle_25: channel: 2870, counts: 6.869999885559082
angle_15: channel: 2912, counts: 124.7300033569336
angle_10: channel: 2920, counts: 173.4499969482422
angle_-15: channel: 2833, counts: 0.9300000071525574


We can see from the above that indeed at the angle 7.5 degrees the most counts per unit time (100s live time) is measured at around bin 2930.

### Fitting a Gaussians for Compton Spectrum & Gamma Decay Peak

In [50]:
import typing
import curve_fitter

filter_7_5_measurement: typing.Callable[[mca_output.MCAOutput], bool] = lambda mca_out: pathlib.Path(mca_out.path).stem == "angle_7.5"
targetless_7_5_measurement = next((mca_out for mca_out in angle_calibration_mca_outs if filter_7_5_measurement(mca_out)), None)

if not targetless_7_5_measurement:
    raise RuntimeError("Measurment not found")

channel_counts = np.array(targetless_7_5_measurement.channel_count_list) / targetless_7_5_measurement.measurement_time
channel_counts_uncertainty = np.sqrt(channel_counts)

MEASUREMENT_LENGTH = len(channel_counts)

channel_index = np.arange(MEASUREMENT_LENGTH, dtype=np.float32)
channel_uncertainty = np.full([MEASUREMENT_LENGTH], 1./np.sqrt(3))

gaussian_peak_model_data = curve_fitter.ModelData(channel_index[2930-200:2930+200], channel_counts_uncertainty[2930-200:2930+200], channel_counts[2930-200:2930+200], channel_counts_uncertainty[2930-200:2930+200])

beta, sd, chi_sq = curve_fitter.odr_fit_gaussian(gaussian_peak_model_data, [2930, 100, 40000])
initial_guess_model = curve_fitter.vectorized_gaussian(2930, 100, 40000)
initial_guess_data = initial_guess_model(np.arange(MEASUREMENT_LENGTH, dtype=np.float32))
gaussian_peak_odr_data = curve_fitter.vectorized_gaussian(beta[0], beta[1], beta[2])(np.arange(MEASUREMENT_LENGTH, dtype=np.float32))
print("Gaussian Peak fitting data: ", beta, sd, chi_sq)

compton_spectrum_model_data = curve_fitter.ModelData(channel_index[600:2400], channel_counts_uncertainty[600:2400], channel_counts[600:2400], channel_counts_uncertainty[600:2400])
first_gaussian_initial_guess = [900 , 380, np.sum(channel_counts[735:925])*2]
second_gaussian_initial_guess = [1980, 360, np.sum(channel_counts[1900:2080])*2]

print(f"Normalization guesses: {np.sum(channel_counts[735:925])*2}, {np.sum(channel_counts[1900:2080])*2}")
beta, sd, chi_sq = curve_fitter.odr_fit_gaussian_sum(compton_spectrum_model_data, first_gaussian_initial_guess+second_gaussian_initial_guess)
print("Compton spectrum fitting data: ", beta, sd, chi_sq)
compton_spectrum_odr_data = curve_fitter.vectorized_two_gaussian_sum(beta)(np.arange(MEASUREMENT_LENGTH, dtype=np.float32))

fig = plotly_utils.generate_scatter_and_line_plot(channel_counts, gaussian_peak_odr_data, compton_spectrum_odr_data)
fig.update_layout(height=800)
fig.show()

Gaussian Peak fitting data:  [ 2927.54656851   104.93651132 46784.93067335] [ 0.11047535  0.10303988 42.759325  ] 0.02108790325893657
Normalization guesses: 5435.26, 5634.0
Compton spectrum fitting data:  [  984.3736844    355.7771356  13854.53620126  1829.36709104
   318.96962047 13092.08414216] [  4.27457214   5.06524425 213.75229392   4.29904569   2.71489828
 179.92232315] 0.058869743912641125
